In [ ]:
from selenium import webdriver
import time
import openpyxl
from selenium.webdriver.common.keys import Keys
import pandas as pd
from bs4 import BeautifulSoup

# 최종 영화 csv 파일로 사용 / id number 있으면 그것도 불러옴
mlist = pd.read_csv('./미재개봉 8점이상.csv')
indexnum = mlist['code']
searching = mlist['name']
# rating = mlist['rating']

try:
    wb = openpyxl.load_workbook("KOBIS_미재개봉.xlsx")
    sheet = wb.active
    print("불러오기 완료")
except:
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.append(["인덱스", "검색어", "영화명", "코드", "분류", "장르", "시간", "관람가", "국가",
                  "개봉일", "재개봉일", "개봉년도", "재개봉년도", "재개봉여부",
                  "총스크린수", "총상영횟수", "총누적매출액", "총누적관객수", "서울시스크린", "전국스크린", "서울시매출액", "전국매출액",
                  "서울시관객수", "전국관객수", "비고", "서울데이터"])
    print("새로운 파일을 만들었습니다.")

driver = webdriver.Chrome("./chromedriver")

for i in range(0, 17178):
    try:
        # 영화 검색
        driver.get("http://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do")
        time.sleep(2)

        search_box = driver.find_element_by_css_selector("input#inp_solrSearch")
        search_box.send_keys(searching[i])
        search_button = driver.find_element_by_css_selector("button.btn_sch")
        search_button.click()

        time.sleep(3)

        # 검색 결과의 첫 번째 영화 클릭
        mname_button = driver.find_element_by_xpath("//a[@href='#none']")
        mname_button.click()
        time.sleep(2)

        #영화코드
        mcode = driver.find_elements_by_css_selector("dl.ovf.cont > dd")[0].text.strip()
        # 검색한 영화와 실제 결과로 나온 영화가 일치하는지 알아보기 위해 수집
        movie_name = driver.find_element_by_css_selector("div.hd_layer strong.tit").text.strip()

        # 상세정보에서 개봉일/재개봉일 수집
        summary_info = driver.find_elements_by_css_selector("dl.ovf.cont > dd")[3].text.strip()
        open_date = driver.find_elements_by_css_selector("dl.ovf.cont > dd")[4].text.strip()

        try:
            reopen = driver.find_element_by_xpath("//dt[text()='재개봉일']")
            reopen_date = driver.find_elements_by_css_selector("dl.ovf.cont > dd")[7].text.strip()
            re_whether = 1
        except:
            reopen_date = ""
            re_whether = 0

        stat_button = driver.find_element_by_xpath("//a[text()='통계정보']")
        stat_button.click()
        time.sleep(10)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 총스크린수, 총상영횟수, 총누적매출액, 총누적관객수 구하기 - 상영타입별 누적통계에서
        try:
            num_list = soup.select("tr.summary > td.tar")
            total_screen = num_list[3].text.strip()
            total_show = num_list[4].text.strip()
            total_revenue = num_list[5].text.strip()
            total_attendance = num_list[6].text.strip()
        # 상영타입별 누적통계 없을 경우 - 0으로 채우기
        except:
            total_screen = 0
            total_show = 0
            total_revenue = 0
            total_attendance = 0

        # "서울시스크린", "서울시매출액", "서울시관객수", "전국스크린", "전국매출액", "전국관객수" 구하기 - 공식통계 정보란
        seoul_list = soup.select(
            ".cont_tab > div:nth-of-type(2) > div:nth-of-type(1) > table > tbody > tr > td[class=tar]")
        try:
            seoul_screen = seoul_list[0].text.strip()
            seoul_revenue = seoul_list[1].text.strip()
            seoul_attendance = seoul_list[2].text.strip()
        except:
            seoul_screen = 0
            seoul_revenue = 0
            seoul_attendance = 0

        try:
            whole_screen = seoul_list[3].text.strip()
            whole_revenue = seoul_list[4].text.strip()
            whole_attendance = seoul_list[5].text.strip()
        except:
            whole_screen = 0
            whole_revenue = 0
            whole_attendance = 0


        sheet.append([indexnum[i], searching[i], movie_name, mcode, summary_info, "", "", "", "",
                      open_date, reopen_date, "", "", re_whether,
                      total_screen, total_show, total_revenue, total_attendance, seoul_screen, whole_screen, seoul_revenue, whole_revenue,
                      seoul_attendance, whole_attendance, "", ""])
        wb.save("KOBIS_미재개봉.xlsx")

    except:
        print(indexnum[i], ',', searching[i], "오류 발생")
        continue

driver.close()
wb.save("KOBIS_미재개봉.xlsx")